# ACT Conversation Steering Demo

This notebook demonstrates the ACT (Affect Control Theory) conversation steering system.

## Features Demonstrated:
1. Setting up identities and modifiers
2. EPA calibration with behavior-based prompts
3. Multi-turn conversation with deflection tracking
4. Deflection controller with PID, decay, and windowing
5. Inauthenticity constraints

## 1. Setup and Imports

In [1]:
import sys
sys.path.insert(0, '../..')

# Core ACT modules
from examples.act.act_core import (
    EPA, ACTCoefficients, get_default_coefficients,
    impression_formation, calculate_deflection, find_optimal_behavior
)

# Identity management
from examples.act.identity_manager import (
    IdentityDatabase, ModifierDatabase, create_identity,
    get_identity_database, get_modifier_database
)

# Calibration
from examples.act.epa_calibration import (
    BehaviorPromptGenerator, LinearRegressionCalibrator,
    CalibrationCoefficients
)

# Steering
from examples.act.conversation_steering import (
    ACTSteeringEngine, DeflectionControllerConfig, ContextMode
)

# Inauthenticity
from examples.act.inauthenticity import (
    calculate_inauthenticity, constrained_optimal_behavior,
    InauthenticityConstraints
)

import numpy as np
import matplotlib.pyplot as plt

print("ACT Steering modules loaded successfully!")

ACT Steering modules loaded successfully!


## 2. Explore Identities and Modifiers

In [2]:
# Load databases
identity_db = get_identity_database()
modifier_db = get_modifier_database()

print(f"Loaded {len(identity_db)} identities and {len(modifier_db)} modifiers\n")

# Look up some identities
for name in ['doctor', 'friend', 'criminal', 'child', 'ceo']:
    identity = identity_db.get(name)
    if identity:
        print(f"{identity.name:15} E={identity.epa.e:+.2f} P={identity.epa.p:+.2f} A={identity.epa.a:+.2f}")

Loaded 968 identities and 660 modifiers

doctor          E=+2.59 P=+2.56 A=+1.29
friend          E=+3.17 P=+1.68 A=+0.93
criminal        E=-3.06 P=+0.11 A=+1.18
child           E=+2.21 P=-1.68 A=+2.39
CEO             E=+0.79 P=+3.01 A=+2.05


In [3]:
# Create modified identities
angry_doctor = create_identity('doctor', ['angry'])
friendly_assistant = create_identity('assistant', ['friendly'])

print(f"Angry Doctor: {angry_doctor.epa}")
print(f"Friendly Assistant: {friendly_assistant.epa}")

Angry Doctor: EPA(e=0.30, p=1.30, a=1.42)
Friendly Assistant: EPA(e=2.45, p=0.83, a=1.03)


## 3. Core ACT Functions

In [4]:
# Load ACT coefficients
coefficients = get_default_coefficients()

# Define identities
assistant_epa = identity_db.get('assistant').epa
customer_epa = identity_db.get('customer').epa

print(f"Assistant: {assistant_epa}")
print(f"Customer: {customer_epa}")

Assistant: EPA(e=1.87, p=-0.24, a=0.99)
Customer: EPA(e=1.70, p=1.22, a=0.90)


In [5]:
# Simulate a customer complaint (negative E, moderate P, high A)
complaint_behavior = EPA(e=-1.5, p=0.5, a=1.8)
print(f"Customer Complaint: {complaint_behavior}")

# Calculate transient impressions after this action
post = impression_formation(
    actor=customer_epa,
    behavior=complaint_behavior,
    obj=assistant_epa,
    coefficients=coefficients
)

print(f"\nAfter customer's complaint:")
print(f"  Customer transient: {post['actor']}")
print(f"  Assistant transient: {post['object']}")

Customer Complaint: EPA(e=-1.50, p=0.50, a=1.80)

After customer's complaint:
  Customer transient: EPA(e=-0.68, p=1.43, a=2.06)
  Assistant transient: EPA(e=0.27, p=-1.47, a=-1.12)


In [6]:
# Find optimal response behavior for assistant
optimal_response = find_optimal_behavior(
    actor_fundamental=assistant_epa,
    object_fundamental=customer_epa,
    actor_transient=post['object'],  # Assistant's transient
    object_transient=post['actor'],   # Customer's transient
    coefficients=coefficients
)

print(f"Optimal Response EPA: {optimal_response}")
print(f"\nInterpretation:")
print(f"  Evaluation: {'positive/kind' if optimal_response.e > 0 else 'negative/critical'}")
print(f"  Potency: {'strong/authoritative' if optimal_response.p > 0 else 'yielding/submissive'}")
print(f"  Activity: {'lively/energetic' if optimal_response.a > 0 else 'calm/passive'}")

Optimal Response EPA: EPA(e=1.00, p=1.34, a=1.31)

Interpretation:
  Evaluation: positive/kind
  Potency: strong/authoritative
  Activity: lively/energetic


## 4. Deflection Tracking

In [7]:
# Calculate deflection if assistant responds with optimal vs suboptimal behavior

# Optimal response
post_optimal = impression_formation(
    actor=post['object'],  # Assistant
    behavior=optimal_response,
    obj=post['actor'],  # Customer
    coefficients=coefficients
)

defl_optimal = (
    calculate_deflection(assistant_epa, post_optimal['actor']) +
    calculate_deflection(customer_epa, post_optimal['object'])
)

# Suboptimal response (e.g., curt reply)
curt_response = EPA(e=-0.5, p=0.8, a=0.2)
post_curt = impression_formation(
    actor=post['object'],
    behavior=curt_response,
    obj=post['actor'],
    coefficients=coefficients
)

defl_curt = (
    calculate_deflection(assistant_epa, post_curt['actor']) +
    calculate_deflection(customer_epa, post_curt['object'])
)

print(f"Deflection with optimal response: {defl_optimal:.3f}")
print(f"Deflection with curt response: {defl_curt:.3f}")
print(f"\nOptimal response reduces deflection by {(defl_curt - defl_optimal):.3f}")

Deflection with optimal response: 8.249
Deflection with curt response: 12.428

Optimal response reduces deflection by 4.180


## 5. Steering Engine Demo

In [1]:
# Create steering engine with PID controller
controller_config = DeflectionControllerConfig(
    enabled=True,
    context_mode=ContextMode.HISTORY,
    window_size=5,
    use_decay=True,
    decay_rate=0.8,
    kp=1.0,
    ki=0.2,
    kd=0.1
)

engine = ACTSteeringEngine(
    agent_identity='assistant',
    user_identity='customer',
    controller_config=controller_config
)

print(f"Engine initialized!")
print(f"Agent: {engine.agent.name} ({engine.agent.epa})")
print(f"User: {engine.user.name} ({engine.user.epa})")

NameError: name 'DeflectionControllerConfig' is not defined

In [ ]:
# Mock EPA reading function for demo
def mock_read_epa(text):
    """Simple heuristic-based EPA reading."""
    e = 0.5 if 'thank' in text.lower() or 'please' in text.lower() else -0.5
    p = 0.8 if 'need' in text.lower() or 'must' in text.lower() else 0.0
    a = 1.0 if 'urgent' in text.lower() or '!' in text else 0.2
    return EPA(e=e, p=p, a=a)

def mock_steer(prompt, target_epa):
    """Mock steering that returns a simple response."""
    return f"I understand your concern. Let me help you with that."

engine.set_read_epa_function(mock_read_epa)
engine.set_steer_function(mock_steer)

print("Mock functions set!")

In [ ]:
# Simulate a conversation
messages = [
    "I need help with my order urgently!",
    "This is frustrating, it's been days!",
    "Thank you for looking into it.",
    "Please let me know when it's resolved."
]

deflections = []

for msg in messages:
    print(f"\nUser: {msg}")
    
    # Process user message
    optimal_epa = engine.process_user_message(msg)
    target_epa = engine.get_adjusted_target(optimal_epa)
    
    print(f"  Optimal EPA: {optimal_epa}")
    print(f"  Adjusted Target: {target_epa}")
    
    # Generate response
    response = engine.generate_response(msg, target_epa)
    actual_epa, deflection = engine.process_response(response, target_epa)
    
    print(f"  Response: {response}")
    print(f"  Deflection: {deflection:.3f}")
    
    deflections.append(deflection)

In [ ]:
# Plot deflection over conversation
plt.figure(figsize=(10, 4))
plt.plot(range(1, len(deflections)+1), deflections, 'o-', linewidth=2, markersize=8)
plt.xlabel('Turn')
plt.ylabel('Deflection')
plt.title('Deflection Over Conversation')
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nFinal Metrics:")
for key, value in engine.get_metrics().items():
    print(f"  {key}: {value:.3f}" if isinstance(value, float) else f"  {key}: {value}")

## 6. Inauthenticity Constraints

In [ ]:
# Compare optimal behavior with and without authenticity constraints
constraints = InauthenticityConstraints(
    max_inauthenticity=3.0,
    authenticity_weight=0.5
)

# Standard optimal (may be inauthentic)
standard_optimal = find_optimal_behavior(
    actor_fundamental=assistant_epa,
    object_fundamental=customer_epa,
    actor_transient=assistant_epa,
    object_transient=customer_epa,
    coefficients=coefficients
)

# Authenticity-constrained optimal
constrained_optimal, metrics = constrained_optimal_behavior(
    actor_fundamental=assistant_epa,
    object_fundamental=customer_epa,
    actor_transient=assistant_epa,
    object_transient=customer_epa,
    coefficients=coefficients,
    constraints=constraints
)

# Compare
std_inauth = calculate_inauthenticity(assistant_epa, standard_optimal)
con_inauth = calculate_inauthenticity(assistant_epa, constrained_optimal)

print(f"Standard Optimal: {standard_optimal}")
print(f"  Inauthenticity: {std_inauth:.3f}")
print(f"\nConstrained Optimal: {constrained_optimal}")
print(f"  Inauthenticity: {con_inauth:.3f}")
print(f"  Total Deflection: {metrics['total_deflection']:.3f}")

## 7. Re-identification Demo

In [ ]:
# Demonstrate re-identification
# When transient impressions deviate significantly, actors may be re-labeled

# Imagine the assistant has been very aggressive
transient_epa = EPA(e=-2.0, p=2.5, a=2.0)

# Find the identity that best matches this transient
closest = identity_db.find_closest(transient_epa, limit=5)

print(f"Transient EPA: {transient_epa}")
print(f"\nClosest matching identities:")
for identity, distance in closest:
    print(f"  {identity.name:20} (distance: {distance:.3f}) {identity.epa}")

## Next Steps

To use this with an actual LLM:

1. **Calibration**: Run `epa_calibration.py` with your model to calibrate EPA readings
2. **Integration**: Set up actual `read_epa` and `steer` functions with your LLM pipeline
3. **Web Demo**: Run `python demo/demo_server.py` to start the browser interface